# Cart-Pole: Linearization and LQR Balncing

## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/drake.html).

In [1]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

# Setup matplotlib.
from IPython import get_ipython
if get_ipython() is not None: get_ipython().run_line_magic("matplotlib", "inline")

In [2]:
# python libraries
import numpy as np
from IPython.display import HTML, display

# underactuated imports
from underactuated import FindResource

## Problem Description
In this problem you will work on the cart-pole system described in [Section 3.2 of the textbook](http://underactuated.csail.mit.edu/acrobot.html#cart_pole).
You will be asked to write down its dynamics in state-space form, linearized it, and then analyze the linearization error.
At the end we will wire up an LQR controller and we will simulate the cart-pole in a series of balancing tasks.

## Dynamics in State-Space Form
Consider the cart-pole system described in [Section 3.2 of the textbook](http://underactuated.csail.mit.edu/acrobot.html#cart_pole).
For the sake of simplicity, in this notebook we fix the following numeric values for its parameters:
- mass of the cart $m_{\text{c}}=1$,
- mass of the pole $m_{\text{p}}=1$,
- length of the pole $l=1$,
- gravity acceleration $g=9.81$.

**Important:** Do not modify/round these parameters, otherwise the autograding code will raise an error.

Using the same convention of the book, we describe the state as $$\mathbf{x} = [x, \theta, \dot{x}, \dot{\theta}]^T,$$ and we let the force on the cart be the control input $\mathbf{u} = [f]$.
Use [equations (16) and (17) from the textbook](http://underactuated.csail.mit.edu/acrobot.html#cart_pole) to derive the state-space model of the cart-pole: $$\dot{\mathbf{x}} = f(\mathbf{x}, \mathbf{u}).$$


In [3]:
# function that given the state x (4d array)
# and the input u (1d array) returns the right
# hand side of the state space dynamics f(x,u)
# (remember that we fixes the cart-pole parameters
# to the values above!)
def f(x, u):
    
    # shortcuts for the cosine and the sine of theta
    # they might be handy
    c = np.cos(x[1])
    s = np.sin(x[1])
    
    # gravity acceleration
    g = 9.81 # do not change
        
    # fill the following matrix
    # (sorry for the one-base counting!)
    f1 = x[2] # modify here
    f2 = x[3] # modify here
    f3 = ( u[0] + s * ( x[3]**2 + 9.81 * c ) ) / ( 1 + s**2 ) # modify here
    f4 = ( -u[0] * c - x[3]**2 * c * s - 2 * 9.81 * s )/( 1 + s**2 ) # modify here
    
    return np.array([f1, f2, f3, f4])

## Dynamics Linearized Around the Unstable Equilibrium
We now approximate the nonlinear dynamics with a linear one.
This will allow us to basic linear control to stabilize (locally) the cart-pole with the pole in the vertical configuration.

We consider the unstable equilibrium state $$\mathbf{x}^* = [0, \pi, 0, 0]^T,$$ with the related equilibrium control input $$\mathbf{u}^* = [0].$$
As in [Section 3.4.1](http://underactuated.csail.mit.edu/acrobot.html#section4), we want to derive a linear model in the from
$$\dot{\bar{\mathbf{x}}} = A_{\text{lin}} \mathbf{\bar{x}} + B_{\text{lin}} \mathbf{\bar{u}},$$
where $\mathbf{\bar{x}} = \mathbf{x}-\mathbf{x}^*$ and $\mathbf{\bar{u}} = \mathbf{u} -\mathbf{u}^*$.

Follow the recipe described in [Section 3.4.1 of the textbook](http://underactuated.csail.mit.edu/acrobot.html#section4) to derive the linearization matrices $A_{\text{lin}}$ and $B_{\text{lin}}$, and implement them in the cell below.

In [4]:
# function that returns the A_lin matrix
def get_A_lin():
    g = 9.81 # do not change
    # fill the matrix below
    A = np.array([
        [0, 0, 1, 0], # modify here
        [0, 0, 0, 1], # modify here
        [0, g, 0, 0], # modify here
        [0, g*2, 0, 0]  # modify here
    ])
    return A
    
# function that returns the B_lin matrix
def get_B_lin():
    # fill the matrix below
    B = np.array([
        [0], # modify here
        [0], # modify here
        [1], # modify here
        [1]  # modify here
    ])
    return B

## Linearization Error
The linear model we have built above is very accurate accurate in the vicinity of the equilibrium point, but can lead to very bad predictions if our state is far away from the equilibrium.

The following function, for a given state $\mathbf{x}$ and control $\mathbf{u}$, evaluates the linearization error:
$$
e(\mathbf{x}, \mathbf{u})
=
\| f(\mathbf{x}, \mathbf{u}) - f_{\text{lin}}(\mathbf{x}, \mathbf{u}) \|,$$
where we defined $f_{\text{lin}}(\mathbf{x}, \mathbf{u}) = A_{\text{lin}} \mathbf{\bar{x}} + B_{\text{lin}} \mathbf{\bar{u}}.$

In [5]:
def f_lin(x,u):
    
    # equilibrium point
    x_star = np.array([0, np.pi, 0, 0])
    u_star = np.array([0])
    
    # linearized dynamics
    x_bar = x - x_star
    u_bar = u - u_star
    A = get_A_lin()
    B = get_B_lin()
    
    return A.dot(x_bar) + B.dot(u_bar)
    
def linearization_error(x, u):
    return np.linalg.norm(f(x,u) - f_lin(x,u))

Use the function above to evaluate the error $e(\mathbf{x}, \mathbf{u})$ in the following 6 conditions:
- $\mathbf{x} = [0, 0.99 \pi, 0, 0]^T$ and $\mathbf{u} = [0]$,
- $\mathbf{x} = [0, 0.9 \pi, 0, 0]^T$ and $\mathbf{u} = [-10]$,
- $\mathbf{x} = [0, 0.85 \pi, 0, 0]^T$ and $\mathbf{u} = [0]$,
- $\mathbf{x} = [0, 0.5 \pi, 0, 0]^T$ and $\mathbf{u} = [0]$,
- $\mathbf{x} = [0, 0, 0, 0]^T$ and $\mathbf{u} = [0]$,
- $\mathbf{x} = [1, \pi, 0, 0]^T$ and $\mathbf{u} = [10]$,

**Attention 1:** For the number $\pi$ use `np.pi`! **Do not** truncate it like $3.14$.

In [6]:
# fill these states with the ones given above
x_list = [
    np.array([0, 0.99 * np.pi, 0, 0]), # modify here
    np.array([0, 0.9 * np.pi, 0, 0]), # modify here
    np.array([0, 0.85 * np.pi, 0, 0]), # modify here
    np.array([0, 0.5 * np.pi, 0, 0]), # modify here
    np.array([0, 0 * np.pi, 0, 0]), # modify here
    np.array([1, np.pi, 0, 0])  # modify here
]

# fill these inputs with the ones given above
u_list = [
    np.array([0]), # modify here
    np.array([-10]), # modify here
    np.array([0]), # modify here
    np.array([0]), # modify here
    np.array([0]), # modify here
    np.array([10])  # modify here
]

# compute linearization errors for all the points above
errors = [linearization_error(x_list[i], u_list[i]) for i in range(6)]

Below we compare these linearization errors with the norm of $\dot{\mathbf{x}}$, i.e.,
$$
\| f(\mathbf{x}, \mathbf{u})\|.$$
Clearly, the smaller is the linearization error with respect to this value, the better is our linear model.

Spend the time you need to convince yourself about this result.
Try to answer the following questions:
- Is our linear approximation valid for all the points we tested?
- Do we expect a linear controller to do a decent job when $\theta = \pi/2$?
- When $\theta$ is different from zero, does the linearization error depend on $\mathbf{u}$?
- Why is the error from the second case bigger than the one from the third, even if the second $\theta$ is closer to $\pi$ than the third?
- What about the position $x$ of the cart? Should it affect the linearization error? If no, why not?

(Questions not graded, do not submit.)

In [7]:
for i, e in enumerate(errors):
    print(f'State = {np.around(x_list[i], decimals=3)}^T')
    print(f'Input = {np.around(u_list[i], decimals=3)}')
    print('Linearization error = {:.3f}'.format(e))
    print('Norm of f(x,u) = {:.3f}\n'.format(np.linalg.norm(f(x_list[i], u_list[i]))))

State = [ 0.    3.11  0.    0.  ]^T
Input = [0]
Linearization error = 0.001
Norm of f(x,u) = 0.688

State = [ 0.     2.827  0.     0.   ]^T
Input = [-10]
Linearization error = 2.354
Norm of f(x,u) = 18.450

State = [ 0.    2.67  0.    0.  ]^T
Input = [0]
Linearization error = 2.289
Norm of f(x,u) = 8.085

State = [ 0.     1.571  0.     0.   ]^T
Input = [0]
Linearization error = 26.054
Norm of f(x,u) = 9.810

State = [ 0.  0.  0.  0.]^T
Input = [0]
Linearization error = 68.913
Norm of f(x,u) = 0.000

State = [ 1.     3.142  0.     0.   ]^T
Input = [10]
Linearization error = 0.000
Norm of f(x,u) = 14.142



## Balancing with LQR Controller
We finally move to the design of the LQR controller.
Drake handles all the linearization process very transparently: no need to get your hands dirty with all the linearization issues we have discussed above!
But it was worth to do it by hand at least once...

Drake can design an LQR controller directly on the nonlinear system obtained by parsing the `.urdf` file!

In [8]:
# pydrake imports
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         LinearQuadraticRegulator, Parser,
                         PlanarSceneGraphVisualizer, Simulator, Linearize)

# underactuated imports
from underactuated import FindResource

First we set a couple of numeric parameters.

In [9]:
# unstable equilibrium point
x_star = [0, np.pi, 0, 0]

# weight matrices for the lqr controller
Q = np.eye(4)
R = np.eye(1)

Then we construct the block diagram with the cart-pole in closed loop with the LQR controller.

In [10]:
# start construction site of our block diagram
builder = DiagramBuilder()

# instantiate the cart-pole and the scene graph
cartpole, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
urdf_path = FindResource('models/undamped_cartpole.urdf')
Parser(cartpole).AddModelFromFile(urdf_path)
cartpole.Finalize()

# set the operating point (vertical unstable equilibrium)
context = cartpole.CreateDefaultContext()
context.get_mutable_continuous_state_vector().SetFromVector(x_star)

# fix the input port to zero and get its index for the lqr function
cartpole.get_actuation_input_port().FixValue(context, [0])
input_i = cartpole.get_actuation_input_port().get_index()

# synthesize lqr controller directly from
# the nonlinear system and the operating point
lqr = LinearQuadraticRegulator(cartpole, context, Q, R, input_port_index=input_i)
lqr = builder.AddSystem(lqr)

# the following two lines are not needed here...
output_i = cartpole.get_state_output_port().get_index()
cartpole_lin = Linearize(cartpole, context, input_port_index=input_i, output_port_index=output_i)

# wire cart-pole and lqr
builder.Connect(cartpole.get_state_output_port(), lqr.get_input_port(0))
builder.Connect(lqr.get_output_port(0), cartpole.get_actuation_input_port())

# add a visualizer and wire it
visualizer = builder.AddSystem(
    PlanarSceneGraphVisualizer(scene_graph, xlim=[-3., 3.], ylim=[-1.2, 1.2], show=False)
)
builder.Connect(scene_graph.get_pose_bundle_output_port(), visualizer.get_input_port(0))

# finish building the block diagram
diagram = builder.Build()

# instantiate a simulator
simulator = Simulator(diagram)
simulator.set_publish_every_time_step(False) # makes sim faster

The following cell contains a function that simulates the closed-loop system and produces a video of the sim.

In [11]:
# function that given the cart-pole initial state
# and the simulation time, simulates the system
# and produces a video
def simulate_and_animate(x0, sim_time=5):
    
    # start recording the video for the animation of the simulation
    visualizer.start_recording()
    
    # reset initial time and state
    context = simulator.get_mutable_context()
    context.SetTime(0.)
    context.SetContinuousState(x0)
    
    # run sim
    simulator.Initialize()
    simulator.AdvanceTo(sim_time)
    
    # stop video
    visualizer.stop_recording()
    
    # construct animation
    ani = visualizer.get_recording_as_animation()
    
    # display animation below the cell
    display(HTML(ani.to_jshtml()))
    
    # reset to empty video
    visualizer.reset_recording()

Now we just run the function we just wrote for all the initial states we analyzed in this notebook, and we look at the result!

In [12]:
# simulate and animate the cart
for x in x_list:
    simulate_and_animate(x)

Was your intuition from the previous analysis correct?
Out of the 6 initial states we considered, which are the states from which the LQR controller is able to recover? (Questions not graded, do not submit.)

In the next cell write (in base zero) the indices of the states from which the system is able to recover (autograded).

In [13]:
system_recovers_from_states = [0,1,2,5] # modify here
print('System recovers from states:')
for i in system_recovers_from_states:
    print(np.around(x_list[i], decimals=2))

System recovers from states:
[ 0.    3.11  0.    0.  ]
[ 0.    2.83  0.    0.  ]
[ 0.    2.67  0.    0.  ]
[ 1.    3.14  0.    0.  ]


## A Final Note
In the middle of the construction of the block diagram above, we have hidden the system `cartpole_lin`.
It has been defined using `Linearize`.
This is the function that `LinearQuadraticRegulator` uses to linearize the plant before solving the Riccati equation. 
Feel free to use the methods `cartpole_lin.A()` and `cartpole_lin.B()` to double check your answer above!

## Autograding
You can check your work by running the following cell:

In [14]:
from underactuated.exercises.acrobot.cartpole_balancing.test_cartpole_balancing import TestCartPoleBalancing
from underactuated.exercises.grader import Grader
Grader.grade_output([TestCartPoleBalancing], [locals()], 'results.json')
Grader.print_test_results('results.json')

ModuleNotFoundError: No module named 'timeout_decorator'